In [3]:
# Set up Dependencies
from progress.bar import IncrementalBar
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import nltk
import json
import csv
import sys
import os
from scipy.stats import linregress
from pprint import pprint

In [5]:
# read the data back into the dataframe from the csv file
input_data_file = os.path.join('data_files','good_merged_data.csv')

# write the DataFrame data to the csv file
movie_df = pd.read_csv(input_data_file)

movie_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Budget,IMDB ID,Revenue
0,Vulgar,2000,R,26 Apr 2002,87 min,"Crime, Drama, Thriller",English,NaN,5.0,5.3,"3,876",tt0120467,movie,NaN,Lions Gate Films,120000,tt0120467,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116 min,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,63.0,6.8,"103,339",tt1206543,movie,"$8,314,341",Relativity Media,22000000,tt1206543,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100 min,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,27.0,5.7,"104,644",tt0200550,movie,NaN,Buena Vista,45000000,tt0200550,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105 min,"Comedy, Family",English,1 win & 4 nominations.,36.0,6.0,"24,453",tt1047540,movie,"$77,253,083",20th Century Fox,25000000,tt1047540,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119 min,"Comedy, Drama, Romance",English,1 nomination.,38.0,6.0,"34,868",tt0350028,movie,"$37,379,556",Buena Vista Pictures,50000000,tt0350028,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104 min,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,54.0,6.6,"199,775",tt5442430,movie,"$30,180,466",Skydance Films,58000000,tt5442430,100541806
596,Filth,2013,R,24 Apr 2014,97 min,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,56.0,7.1,"98,589",tt1450321,movie,"$23,145",Magnet Releasing,5000000,tt1450321,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95 min,"Biography, Comedy, Crime","English, Spanish",1 nomination.,47.0,5.8,"48,784",tt2461150,movie,"$17,356,894",Relativity Media,25000000,tt2461150,29200000
598,Planet 51,2009,PG,20 Nov 2009,91 min,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,39.0,6.0,"49,599",tt0762125,movie,"$42,194,060",Sony Pictures/TriStar Pictures,70000000,tt0762125,104945765


In [8]:
movie_df = movie_df.replace({'Not Rated':'Unrated','UNRATED':'Unrated'})

In [9]:
movie_df['Rated'].value_counts()

R          282
PG-13      204
PG          73
Unrated     13
G           13
TV-MA        1
Name: Rated, dtype: int64

In [11]:
movie_df['Runtime'].value_counts()

98 min     21
97 min     21
106 min    19
99 min     18
96 min     18
           ..
79 min      1
81 min      1
148 min     1
172 min     1
7 min       1
Name: Runtime, Length: 94, dtype: int64

In [16]:
movie_df['Runtime'] = movie_df.Runtime.str.replace(' min' , '')
movie_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Budget,IMDB ID,Revenue
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.0,5.3,"3,876",tt0120467,movie,NaN,Lions Gate Films,120000,tt0120467,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,63.0,6.8,"103,339",tt1206543,movie,"$8,314,341",Relativity Media,22000000,tt1206543,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,27.0,5.7,"104,644",tt0200550,movie,NaN,Buena Vista,45000000,tt0200550,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,36.0,6.0,"24,453",tt1047540,movie,"$77,253,083",20th Century Fox,25000000,tt1047540,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,38.0,6.0,"34,868",tt0350028,movie,"$37,379,556",Buena Vista Pictures,50000000,tt0350028,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,54.0,6.6,"199,775",tt5442430,movie,"$30,180,466",Skydance Films,58000000,tt5442430,100541806
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,56.0,7.1,"98,589",tt1450321,movie,"$23,145",Magnet Releasing,5000000,tt1450321,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,47.0,5.8,"48,784",tt2461150,movie,"$17,356,894",Relativity Media,25000000,tt2461150,29200000
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,39.0,6.0,"49,599",tt0762125,movie,"$42,194,060",Sony Pictures/TriStar Pictures,70000000,tt0762125,104945765


In [25]:
movie_df['Runtime'] = movie_df['Runtime'].astype('float64')

In [26]:
movie_runtime_df = movie_df.loc[movie_df['Runtime'] >= 60]
movie_runtime_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Budget,IMDB ID,Revenue
0,Vulgar,2000,R,26 Apr 2002,87.0,"Crime, Drama, Thriller",English,NaN,5.0,5.3,"3,876",tt0120467,movie,NaN,Lions Gate Films,120000,tt0120467,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116.0,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,63.0,6.8,"103,339",tt1206543,movie,"$8,314,341",Relativity Media,22000000,tt1206543,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100.0,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,27.0,5.7,"104,644",tt0200550,movie,NaN,Buena Vista,45000000,tt0200550,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105.0,"Comedy, Family",English,1 win & 4 nominations.,36.0,6.0,"24,453",tt1047540,movie,"$77,253,083",20th Century Fox,25000000,tt1047540,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119.0,"Comedy, Drama, Romance",English,1 nomination.,38.0,6.0,"34,868",tt0350028,movie,"$37,379,556",Buena Vista Pictures,50000000,tt0350028,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104.0,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,54.0,6.6,"199,775",tt5442430,movie,"$30,180,466",Skydance Films,58000000,tt5442430,100541806
596,Filth,2013,R,24 Apr 2014,97.0,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,56.0,7.1,"98,589",tt1450321,movie,"$23,145",Magnet Releasing,5000000,tt1450321,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95.0,"Biography, Comedy, Crime","English, Spanish",1 nomination.,47.0,5.8,"48,784",tt2461150,movie,"$17,356,894",Relativity Media,25000000,tt2461150,29200000
598,Planet 51,2009,PG,20 Nov 2009,91.0,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,39.0,6.0,"49,599",tt0762125,movie,"$42,194,060",Sony Pictures/TriStar Pictures,70000000,tt0762125,104945765


In [32]:
movie_runtime_df['Runtime'] = movie_runtime_df['Runtime'].astype('int64')

C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
movie_runtime_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Budget,IMDB ID,Revenue
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.0,5.3,"3,876",tt0120467,movie,NaN,Lions Gate Films,120000,tt0120467,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,63.0,6.8,"103,339",tt1206543,movie,"$8,314,341",Relativity Media,22000000,tt1206543,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,27.0,5.7,"104,644",tt0200550,movie,NaN,Buena Vista,45000000,tt0200550,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,36.0,6.0,"24,453",tt1047540,movie,"$77,253,083",20th Century Fox,25000000,tt1047540,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,38.0,6.0,"34,868",tt0350028,movie,"$37,379,556",Buena Vista Pictures,50000000,tt0350028,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,54.0,6.6,"199,775",tt5442430,movie,"$30,180,466",Skydance Films,58000000,tt5442430,100541806
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,56.0,7.1,"98,589",tt1450321,movie,"$23,145",Magnet Releasing,5000000,tt1450321,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,47.0,5.8,"48,784",tt2461150,movie,"$17,356,894",Relativity Media,25000000,tt2461150,29200000
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,39.0,6.0,"49,599",tt0762125,movie,"$42,194,060",Sony Pictures/TriStar Pictures,70000000,tt0762125,104945765


In [35]:
movie_clean_df = movie_runtime_df.loc[movie_runtime_df['Budget'] > 10000]
movie_clean_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice,Production,Budget,IMDB ID,Revenue
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.0,5.3,"3,876",tt0120467,movie,NaN,Lions Gate Films,120000,tt0120467,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,63.0,6.8,"103,339",tt1206543,movie,"$8,314,341",Relativity Media,22000000,tt1206543,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,27.0,5.7,"104,644",tt0200550,movie,NaN,Buena Vista,45000000,tt0200550,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,36.0,6.0,"24,453",tt1047540,movie,"$77,253,083",20th Century Fox,25000000,tt1047540,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,38.0,6.0,"34,868",tt0350028,movie,"$37,379,556",Buena Vista Pictures,50000000,tt0350028,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,54.0,6.6,"199,775",tt5442430,movie,"$30,180,466",Skydance Films,58000000,tt5442430,100541806
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,56.0,7.1,"98,589",tt1450321,movie,"$23,145",Magnet Releasing,5000000,tt1450321,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,47.0,5.8,"48,784",tt2461150,movie,"$17,356,894",Relativity Media,25000000,tt2461150,29200000
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,39.0,6.0,"49,599",tt0762125,movie,"$42,194,060",Sony Pictures/TriStar Pictures,70000000,tt0762125,104945765


In [37]:
movie_clean_df.drop(columns=['BoxOffice','Metascore','IMDB ID'], inplace=True)
movie_clean_df

C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.3,"3,876",tt0120467,movie,Lions Gate Films,120000,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,6.8,"103,339",tt1206543,movie,Relativity Media,22000000,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,5.7,"104,644",tt0200550,movie,Buena Vista,45000000,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,6.0,"24,453",tt1047540,movie,20th Century Fox,25000000,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,6.0,"34,868",tt0350028,movie,Buena Vista Pictures,50000000,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,6.6,"199,775",tt5442430,movie,Skydance Films,58000000,100541806
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,7.1,"98,589",tt1450321,movie,Magnet Releasing,5000000,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,5.8,"48,784",tt2461150,movie,Relativity Media,25000000,29200000
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,6.0,"49,599",tt0762125,movie,Sony Pictures/TriStar Pictures,70000000,104945765


In [49]:
movie_clean_df[movie_clean_df['Production'].isnull()==True]

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue
46,Save the Last Dance,2001,PG-13,12 Jan 2001,112,"Drama, Music, Romance",English,6 wins & 8 nominations.,6.2,"55,963",tt0206275,movie,NaN,13000000,91038276
115,The Prodigy,2019,R,08 Feb 2019,92,"Horror, Thriller",English,1 win & 1 nomination.,5.8,"21,353",tt4504044,movie,NaN,6000000,21150265
117,K-12,2019,Unrated,05 Sep 2019,96,"Fantasy, Horror, Musical",English,NaN,6.4,"2,055",tt10521814,movie,NaN,50000,359377
145,Billy Yeager the Ineffable Enigma,2016,NaN,10 Jul 2016,102,Documentary,English,NaN,8.3,11,tt5855378,movie,NaN,165000,2350000
169,Sherrybaby,2006,R,09 Feb 2007,96,Drama,English,Nominated for 1 Golden Globe. Another 10 wins ...,6.6,"11,251",tt0423169,movie,NaN,2000000,622806
184,The Ghosts of Johnson Woods,2015,NaN,01 Nov 2015,80,"Horror, Thriller",English,NaN,5.0,29,tt4374502,movie,NaN,60000,3145
293,The Wicker Man,2006,PG-13,01 Sep 2006,102,"Horror, Mystery, Thriller",English,2 wins & 7 nominations.,3.7,"62,883",tt0450345,movie,NaN,40000000,38755073
337,Richard Jewell,2019,R,13 Dec 2019,131,"Biography, Crime, Drama",English,Nominated for 1 Oscar. Another 6 wins & 11 nom...,7.5,"40,846",tt3513548,movie,NaN,45000000,33904605
361,Mary Jane's Not a Virgin Anymore,1998,NaN,22 Mar 1998,98,"Drama, Romance",English,NaN,6.4,117,tt0117001,movie,NaN,50000,27
379,I Saw the Light,2015,R,01 Apr 2016,123,"Biography, Drama, Music, Romance",English,NaN,5.7,"6,097",tt1490785,movie,NaN,13000000,1646788


In [50]:
movie_clean_df['Production'].fillna(value='Unknown', inplace=True)

C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [51]:
len(movie_clean_df[movie_clean_df['Production'].isna()==True])

0

In [52]:
movie_clean_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.3,"3,876",tt0120467,movie,Lions Gate Films,120000,14904
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,6.8,"103,339",tt1206543,movie,Relativity Media,22000000,15400000
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,5.7,"104,644",tt0200550,movie,Buena Vista,45000000,113916474
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,6.0,"24,453",tt1047540,movie,20th Century Fox,25000000,119772232
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,6.0,"34,868",tt0350028,movie,Buena Vista Pictures,50000000,49718611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,6.6,"199,775",tt5442430,movie,Skydance Films,58000000,100541806
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,7.1,"98,589",tt1450321,movie,Magnet Releasing,5000000,9114264
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,5.8,"48,784",tt2461150,movie,Relativity Media,25000000,29200000
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,6.0,"49,599",tt0762125,movie,Sony Pictures/TriStar Pictures,70000000,104945765


In [53]:
movie_clean_df['ROI%'] = round(((movie_clean_df['Revenue'] - movie_clean_df['Budget']) / movie_clean_df['Budget']) * 100,2)
movie_clean_df

C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue,ROI%
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.3,"3,876",tt0120467,movie,Lions Gate Films,120000,14904,-87.58
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,6.8,"103,339",tt1206543,movie,Relativity Media,22000000,15400000,-30.00
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,5.7,"104,644",tt0200550,movie,Buena Vista,45000000,113916474,153.15
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,6.0,"24,453",tt1047540,movie,20th Century Fox,25000000,119772232,379.09
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,6.0,"34,868",tt0350028,movie,Buena Vista Pictures,50000000,49718611,-0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,6.6,"199,775",tt5442430,movie,Skydance Films,58000000,100541806,73.35
596,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,7.1,"98,589",tt1450321,movie,Magnet Releasing,5000000,9114264,82.29
597,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,5.8,"48,784",tt2461150,movie,Relativity Media,25000000,29200000,16.80
598,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,6.0,"49,599",tt0762125,movie,Sony Pictures/TriStar Pictures,70000000,104945765,49.92


In [55]:
output_data_file = os.path.join('data_files','good_merged_data.csv')

# write the DataFrame data to the csv file
movie_clean_df.to_csv(output_data_file, index=False)